# Demo Scenario 2
> Validate Demonstrate Scenario 2 on a notebook

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| eval: false

from llmcam.downloader import *
from llmcam.dtcam import *
from llmcam.file_manager import *
from llmcam.fn_to_fc import *
from llmcam.oas_to_requests import *
from llmcam.store import *
from llmcam.yolo import *
from llmcam.plotting import *

In [ ]:
#| eval: false

def update_tools(
    services: list # list of services
):
    """Extarct services from ToolBox and integrate to current tools"""
    new = extract_tools_from_services(services)
    for tool in new:
        tools.append(tool)

tools = [tool_schema(fn) for fn in [update_tools, download_file, cap, add_api_tools, plot_object]] + extract_tools_from_services(['ytube_live'])


In [ ]:
#| eval: false

messages = form_msgs([
    ("system", "You are a helpful system administrator. Use the supplied tools to help the user."),
    ("user", "What can LLMCAM do?"),
    ("user", "Here is the live feed for Santa Claus Village: https://www.youtube.com/watch?v=Cp4RRAEgpeU"),
    ("user", "How many people are there currently?"),
    ("user", "Plot the change in the number of people over the last 3 hours. If you don't have the data, capture the images at the last 3 hours."),
    ("user", "It seems you capture the current live moment multiple time. Can't you access the live at 11pm, 12pm, and 1am then plot the number of people?"),
    ("user", "Notify me when there are fewer than 10 people.")
])

complete(messages, tools)
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
What can LLMCAM do?
>> Assistant:
LLMCAM offers a variety of capabilities related to handling and analyzing images, video, and API
integration. Here are some of its functionalities:  1. **Image and Video Capture:**    - Capture
images from specific locations or contexts using a camera.    - Capture frames from YouTube Live and
save them locally.  2. **File Download:**    - Download files from specified URLs and save them to
desired paths.  3. **Image Analysis:**    - Analyze images to gather quantitative information using
advanced models like GPT-4 Vision.  4. **Plotting and Detection:**    - Plot the number of specified
objects throughout multiple images using object detection techniques like YOLO or GPT.  5. **API
Integration:**    - Add API tools to a toolbox, enabling interactions with specific APIs by
providing service names and base URLs.  6. **Tool Management:**    - Update or in

1. User: What Function Calls (FCs) do you have?
2. Assistant: I can monitor YouTube Live feeds and integrate with various services to provide insights and notifications.
3. User: Here’s the YouTube Live feed for Santa Claus Village.
4. Assistant: The YouTube Live feed for Santa Claus Village has been registered.
5. User: How many people are currently there?
6. Assistant: There are currently 52 people at Santa Claus Village.
7. User: Plot the change in the number of people at Santa Claus Village over the last 3 hours.
8. Assistant: [Generates and displays a bar chart showing the trend in people count over the last 3 hours.]
9. User: Notify me if there are fewer than 10 people.
10. Assistant: Notification rule set. I’ll alert you when the number of people at Santa Claus Village drops below 10.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()